In [0]:
# Gold スキーマ設定
catalog_name = "users"
schema = "yukiteru_koide"

# スキーマ切り替え
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {schema}")


In [0]:
# Silver テーブル読み込み
df = spark.table(f"{catalog_name}.{schema}.transactions_enriched")

from pyspark.sql.functions import avg, count, when, col

# 年齢層を追加
df = df.withColumn("age_group", when(col("age") < 30, "Under 30")
                               .when(col("age") < 50, "30-49")
                               .otherwise("50+"))

segment_sales = (
    df
    .groupBy("age_group", "gender", "membership")
    .agg(
        avg("total_price").alias("avg_spend"),
        count("transaction_id").alias("num_transactions")
    )
)

segment_sales.write.format("delta").mode("overwrite").saveAsTable("customer_segment_spending")
